In [1]:
import pandas as pd
from geopy.geocoders import Photon
from unidecode import unidecode
import unidecode

# RUTAS

### Justificación

Este notebook tiene como principal objetico el de organizar un data freme con las geo referencias geograficas  de las ciudades y puntos representativos que integran los corredores de migrantes y sus diferentes combinaciones.

La metodologia de trabajo consiste en:

1: Recopilar de diversas fuentes de información un listado con las ciudades que componen las rutas migratorias de acuerdo con autores principales en el tema que incluyen a  Rodolfo Casillas, CNDH y otros cuyas referencias se utilizan en la mayoria de los informes informativos y trabajos de investigacion sobre la Migracion en Transito Irregular en Mexico.

2: Preparar ese listado en CSV, que posteriormente sea re procesado para cumplir con los requerimientos de las herramientas de traceo de ArcGis.

3: Revisar tutoriales y definir requerimientos:

4: Adaptar CSV

5: Proyectar



In [28]:
# Cargar el archivo CSV
datos_csv = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/1. rutas_raw/Ciudades_Rutas.csv')


In [29]:
#Quitamos acentos de todos lados
def quitar_acentos(valor):
    """Función para quitar acentos de una cadena de texto."""
    if isinstance(valor, str):  # Solo aplicar a cadenas de texto
        return unidecode.unidecode(valor)  # Corrección aquí
    return valor

# Aplicamos la función quitar_acentos a todas las columnas del DataFrame
for columna in datos_csv.columns:
    datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)


In [34]:
datos_csv.shape

(117, 5)

In [33]:
datos_csv.head()

,ciudad,etiqueta,ruta,latitud,longitud
0,"Puebla, Puebla",NaN,NaN,NaN,NaN
1,"Tlaxcala, Tlaxcala",NaN,NaN,NaN,NaN
2,"Lecheria, Estado de Mexico",NaN,NaN,NaN,NaN
3,"Tijuana, Baja California",NaN,NaN,NaN,NaN
4,"Mexicali, Baja California",NaN,NaN,NaN,NaN


#### Encontrar y quitar repetidos

Columna: [Referencia Geográfica]

In [32]:
# Eliminar duplicados basados en la columna 'Referencia Geográfica'
datos_csv = datos_csv.drop_duplicates(subset='ciudad', keep='first')



In [35]:
# Crear una nueva columna 'Estado' extrayendo el texto después de la coma
datos_csv['estado'] = datos_csv['ciudad'].apply(lambda x: x.split(', ')[1] if ', ' in x else '')


In [36]:
from unidecode import unidecode as quitar_acentos

# Ahora aplicar la función corregida a las columnas 'Ciudad' y 'Estado'
#datos_csv['ciudad'] = datos_csv['ciudad'].apply(quitar_acentos)
#datos_csv['Estado'] = datos_csv['Estado'].apply(lambda x: quitar_acentos(x) if pd.notnull(x) else x)


In [37]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
def asignar_ruta(estado):
    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
    if estado in estados_ruta_golfo:
        return 'Ruta del Golfo'
    elif estado in estados_ruta_pacifico:
        return 'Ruta del Pacifico'
    elif estado in estados_ruta_centro:
        return 'Ruta Centro'
    elif estado in ruta_sur:
        return 'Ruta Sur'
    else:
        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

In [38]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
column_order = ['ciudad', 'estado', 'ruta', 'latitud', 'longitud']

# Aplicar el nuevo orden de columnas al DataFrame
datos_csv = datos_csv[column_order]

### Geolocate Addresses

In [43]:
# Inicializar el geolocalizador de Photon
geolocator = Photon(user_agent="measurement", timeout=10)

# Definir la función para obtener latitud y longitud
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:  # Verificar si se encontró la ubicación
            return location.latitude, location.longitude
        else:
            return '', ''  # Devolver cadenas vacías si no se encuentra la ubicación
    except AttributeError:
        return '', ''

#Aplicar la función a la columna 'Ciudad' y dividir los resultados en las columnas 'Latitud' y 'Longitud'
datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [1]:
datos_csv.head(6)
#datos_csv['ruta']unique values

NameError: name 'datos_csv' is not defined

In [40]:
# Obtenemos los valores únicos de la columna 'ruta'
valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
print(valores_unicos_ruta)


['Ruta Centro' 'Ruta del Pacifico' 'Ruta del Golfo' 'Ruta Sur']


In [27]:
# Filtramos el DataFrame para obtener solo las filas con 'Ruta no identificada'
#datos_ruta_no_identificada = datos_csv[datos_csv['ruta'] == 'Ruta no identificada']

# Mostramos las primeras filas del nuevo DataFrame para verificar
#print(datos_ruta_no_identificada.head())


                ciudad estado                  ruta    latitud    longitud
116  Saltillo,Coahuila         Ruta no identificada  25.423043 -100.992751


### Agregamos columna 'etiqueta' con el Codigo de Ruta, para trabajar con nuestras rutas codificadas.

Tenemos que hacer que se codifique cada 'ciudad' dentro del df 'datos_csv', deacuerdo a su latitud y longitud y ala ruta ala que pertenece.

Cada 'ciudad' deacuerdo a su posicion en el mapa y a la columna 'ruta' a la que pertenece;

Ruta Sur = S
Ruta del Pacifico = P
Ruta Centro = C
Ruta del Golfo = G

ademas de un numero deacuerdo a su posicion en el mapa de manera diagonal de izquierda a derecha de abajo a arriba, por ejemplo:

Tapachula Chiapas ['etiqueta']= 'S1' (por que seria la primer posicion de izquierda a derecha de abajo a arriba., la siguiente ciudad de abajo arriba y de izquierda a derecha seria 'S2' y asi sucesivamente.

¿Como podemos lograr esto? estaba pensando que usaramos una columna para sacar su posicion en diagonal que nos de un numero usando ambas latitud y longitud y este numero nos de de forma ascendente su posicion en el mapa. Podria funcionar? que alternativas o metodos tienes para hacer que esto funcione?

In [48]:
# Paso 1: Calculamos la posición diagonal como la suma de latitud y longitud
datos_csv['posicion_diagonal'] = datos_csv['latitud'] + datos_csv['longitud']

# Paso 2: Mapeamos las rutas a sus códigos correspondientes
mapeo_rutas = {
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
}
datos_csv['codigo_ruta'] = datos_csv['ruta'].map(mapeo_rutas)

# Paso 3: Ordenamos por ruta y posición diagonal, luego asignamos etiquetas numeradas dentro de cada grupo de ruta
datos_csv = datos_csv.sort_values(['codigo_ruta', 'posicion_diagonal'])

# Aquí generamos una nueva columna 'etiqueta' que combina el código de ruta con un número ascendente dentro de cada ruta
datos_csv['etiqueta'] = datos_csv.groupby('codigo_ruta').cumcount() + 1
datos_csv['etiqueta'] = datos_csv['codigo_ruta'] + datos_csv['etiqueta'].astype(str)

# Opcionalmente, podemos eliminar la columna 'Posicion_Diagonal' si ya no es necesaria
#datos_csv.drop('posicion_diagonal', axis=1, inplace=True)


In [51]:
datos_csv.tail(60)

,ciudad,estado,ruta,latitud,longitud,codigo_ruta,etiqueta,posicion_diagonal
75,"Cabo San Lucas, Baja California Sur",Baja California Sur,Ruta del Pacifico,22.893888,-109.920060,P,P2,-87.026172
72,"Pichilingue, Baja California Sur",Baja California Sur,Ruta del Pacifico,24.118173,-110.315746,P,P3,-86.197573
73,"La Paz, Baja California Sur",Baja California Sur,Ruta del Pacifico,24.161995,-110.315853,P,P4,-86.153858
124,"Guerrero Negro, Baja California Sur",Baja California Sur,Ruta del Pacifico,27.970951,-114.036389,P,P5,-86.065438
79,"Barra de Navidad, Jalisco",Jalisco,Ruta del Pacifico,19.205644,-104.682090,P,P6,-85.476445
69,"Loreto, Baja California Sur",Baja California Sur,Ruta del Pacifico,26.009791,-111.345252,P,P7,-85.335461
80,"Manzanillo, Colima",Colima,Ruta del Pacifico,19.050626,-104.315996,P,P8,-85.265370
127,"Santa Rosalia, Baja California Sur",Baja California Sur,Ruta del Pacifico,27.336428,-112.268046,P,P9,-84.931618
128,"Punta Chivato, Baja California Sur",Baja California Sur,Ruta del Pacifico,27.068680,-111.963730,P,P10,-84.895050
126,"Isla San Marcos, Baja California Sur",Baja California Sur,Ruta del Pacifico,27.187970,-112.080065,P,P11,-84.892095


# Guardamos CSV general.

In [50]:
# Especificamos la ruta donde queremos guardar el archivo CSV
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/2. rutas_clase/Ciudades_Rutas_Clase.csv'

# Guardamos el DataFrame en el archivo CSV en la ruta especificada
datos_csv.to_csv(ruta_archivo, index=False)

# Imprimimos un mensaje de confirmación
print("El DataFrame ha sido guardado exitosamente en:", ruta_archivo)


El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/3. Rutas/2. rutas_clase/Ciudades_Rutas_Clase.csv
